In [1]:
!git clone https://@github.com/marcin119a/sat-experiments.git
%cd sat-experiments

Cloning into 'sat-experiments'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 231 (delta 52), reused 0 (delta 0), pack-reused 137
Receiving objects: 100% (231/231), 11.74 MiB | 17.12 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/sat-experiments


##### Kompilacja akmaxsat

In [6]:
%cd /content/sat-experiments/akmaxsat_1.1
!make
%cd /content/sat-experiments

In [3]:
%%capture
!pip install -r requirements.txt

### Pobranie projektu generujacego zbiór danych 

In [14]:
!git clone https://github.com/satproject/neuralheuristicsforsat.git

Cloning into 'neuralheuristicsforsat'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 76 (delta 23), reused 6 (delta 1), pack-reused 25
Unpacking objects: 100% (76/76), done.
/content/sat-experiments


### postawowe parametry symulatora SIMCim

In [9]:
hyperparameters = {
    #'class_weight': hp.choice('class_weight', [None, 'balanced']), @TODO
    'N': 400,
    'sigma': 0.2,
    'dt': 0.3,
    'zeta': 1,
    'attempt_num': 1000,
    'alpha': 0.2,
    'complexity': 50,
    'seed': 1000,
    'observations': 1000 #batch_size
}

### Wygenerowanie zbioru danych D-SAT(20)

In [7]:
%%capture
!python /content/sat-experiments/neuralheuristicsforsat/dump_dataset_2sat.py -o 10000 -c 20 -j 10000

### Uruchomienie algorytmów akmaxsat SimCim i brute-force dla D-SAT(20)

In [19]:
import os
import tensorflow as tf
import itertools
from neuralheuristicsforsat.helpers import chunkIt, save_to_csv
 
from maxsat import (
    akmaxsat_format_to_list, akmaxsat_run, function_from_akmaxsat, brute_force_2_max_sat, dump, transform
)
import time
from SIMCIM import SIMCim
import csv

params = hyperparameters
n = 20 #ilosc zmiennych
tfrecord_location = '/content/sat-experiments/sr_20/train_10000_sr_20.tfrecord' #sciezka do pliku D-SAT(50)

record_iterator = tf.python_io.tf_record_iterator(path=tfrecord_location)
preds = []
batch_size = params['observations']


from importlib import reload

train_set = {'cnf': list(), 'sat': list()}
sim_results = list()

for string_record in itertools.islice(record_iterator, 100): # dla 100 formuł uruchamiamy eksperyment
    example = tf.train.Example()
    example.ParseFromString(string_record) # wczytujemy dataset D-(20)

    m = len(example.features.feature["inputs"].float_list.value) // 2 # ilosc klazul w formule 2CNF

    inputs = chunkIt(example.features.feature["inputs"].float_list.value, m) # fCNFormuly w postaci 
    train_set['cnf'].append(inputs)
    targ = int(example.features.feature["sat"].float_list.value[0])
    train_set['sat'].append(targ)

    start = time.time()
    cnf = transform(inputs)
    dump(inputs, n, m, round(start,2)) #zapisanie do pliku dla algorytmu akmaxsat

    s = '/content/sat-experiments/out/{0}.txt'.format(round(start,2)) # nazwa pliku wejscia dla algorytmu akmaxsat
    output = akmaxsat_run('/content/sat-experiments/akmaxsat_1.1/akmaxsat',s) # uruchomienie algorytmu akmaxsat dla tego pliku

      
    literals = set()
    for conj in cnf:
            for disj in conj:
                literals.add(disj[0])
    check_cnf = [bool(disj.intersection(output)) for disj in cnf] #sprawdzenie ile klauzul zostało spełnionych
    MAX_akmaxsat = check_cnf.count(True)
    if targ == 0: 
      vars, max_2sat = brute_force_2_max_sat(cnf) # uruchomienie algorytmu bruteforce tylko dla formuł które są UNSAT=0
    else:
      max_2sat = m 
    
    sim_result = SIMCim(params, n, m, inputs)  # predykcja symulatora SIMCim

    end = time.time()
    # zapisanie wyników do pliku csv
    save_to_csv('max_sat_100.csv', [[m, m / n, m + sim_result, MAX_akmaxsat, max_2sat , targ, end - start]], n)